<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Обычное-обучение" data-toc-modified-id="Обычное-обучение-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обычное обучение</a></span></li><li><span><a href="#Upsampling-обучение" data-toc-modified-id="Upsampling-обучение-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Upsampling обучение</a></span></li><li><span><a href="#Downsampling-обучение" data-toc-modified-id="Downsampling-обучение-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Downsampling обучение</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
pip install -U -q imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
    from imblearn.pipeline import Pipeline
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Подготовка

In [3]:
try:
    data = pd.read_csv('C:/datasets/toxic_comments.csv', index_col=[0])
except:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0])    
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

<div class="alert alert-block alert-info">
<b>Комментарий:</b> Наблюдается большой дизбаланс классов
</div>

In [5]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [6]:
wnl = WordNetLemmatizer()
def lemmatize_text(text):
    new_text = " ".join(re.sub(r'[^a-zA-Z]', ' ', text).split())
    return " ".join([wnl.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(new_text)])

<div class="alert alert-block alert-info">
<b>Комментарий:</b> Попимо привычной лемматизации добавляем pos-tag для опеределения части речи, чтобы привести слово в более точное форму.
</div>

In [7]:
%%time
data['lemm_text'] = data['text'].apply(lemmatize_text)

CPU times: user 16min 39s, sys: 1min 38s, total: 18min 17s
Wall time: 18min 18s


In [8]:
data['lemm_text']

0         Explanation Why the edits make under my userna...
1         D aww He match this background colour I m seem...
2         Hey man I m really not try to edit war It s ju...
3         More I can t make any real suggestion on impro...
4         You sir be my hero Any chance you remember wha...
                                ...                        
159446    And for the second time of ask when your view ...
159447    You should be ashamed of yourself That be a ho...
159448    Spitzer Umm there no actual article for prosti...
159449    And it look like it be actually you who put on...
159450    And I really don t think you understand I come...
Name: lemm_text, Length: 159292, dtype: object

In [9]:
features_train, features_test, target_train, target_test = train_test_split(data['lemm_text'], data['toxic'], 
                                                                            test_size=0.2, stratify=data['toxic'])

In [10]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf_train = TfidfVectorizer(stop_words=list(stopwords))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
print('Отношение классов целевого признака:',target_train[target_train == 0].shape[0]
      /target_train[target_train == 1].shape[0])

Отношение классов целевого признака: 8.841146034442815


## Обучение

### Обычное обучение

In [12]:
stratified_kfold = StratifiedKFold(n_splits=5)

In [13]:
%%time
m_log_r = LogisticRegression(random_state=12345)
pipeline_lr = Pipeline([('tfid', count_tf_idf_train), ('model', m_log_r)])
scores_log_r = cross_val_score(pipeline_lr, features_train, target_train, scoring='f1', cv=stratified_kfold, n_jobs=-1)
print('LogisticRegression f1 score:', scores_log_r.mean())

LogisticRegression f1 score: 0.7245232469457559
CPU times: user 1min 57s, sys: 1min 51s, total: 3min 49s
Wall time: 3min 49s


In [14]:
%%time
m_lgb = lgb.LGBMClassifier(random_state=12345)
pipeline_lgb = Pipeline([('tfid', count_tf_idf_train), ('model', m_lgb)])
scores_lgb = cross_val_score(pipeline_lgb, features_train, target_train, scoring='f1', cv=stratified_kfold, n_jobs=-1)
print('LGBMClassifier f1 score:', scores_lgb.mean())

LGBMClassifier f1 score: 0.7424863508327213
CPU times: user 13min 36s, sys: 0 ns, total: 13min 36s
Wall time: 13min 43s


### Upsampling обучение

<div class="alert alert-block alert-info">
<b>Комментарий:</b> Так как у нас наблюдается дизбаланс классов, попробуем методику upsampling
</div>

In [15]:
smote = SMOTE(sampling_strategy = 0.3, random_state = 12345)

In [16]:
f1_score_log_r_upsampling = []
m_log_r_upsampling = LogisticRegression(random_state=12345)
pipeline_log_r_upsampling = Pipeline([('tfid', count_tf_idf_train), ('upsampling',smote), ('model', m_log_r_upsampling)])
for train, test in stratified_kfold.split(features_train,target_train):
    model = pipeline_log_r_upsampling.fit(features_train.iloc[train],target_train.iloc[train])
    prediction = model.predict(features_train.iloc[test])
    f1_score_log_r_upsampling.append(f1_score(target_train.iloc[test], prediction))
print("f1: ",np.mean(f1_score_log_r_upsampling))

f1:  0.7705912904694763


In [17]:
%%time
f1_score_lgb_upsampling = []
m_lgb_upsampling = lgb.LGBMClassifier(random_state=12345)
pipeline_lgb_upsampling = Pipeline([('tfid', count_tf_idf_train), ('upsampling',smote), ('model', m_lgb_upsampling)])
for train, test in stratified_kfold.split(features_train,target_train):
    model = pipeline_lgb_upsampling.fit(features_train.iloc[train],target_train.iloc[train])
    prediction = model.predict(features_train.iloc[test])
    f1_score_lgb_upsampling.append(f1_score(target_train.iloc[test], prediction))
print("f1: ",np.mean(f1_score_lgb_upsampling))

f1:  0.7509367021251275
CPU times: user 19min 23s, sys: 0 ns, total: 19min 23s
Wall time: 19min 31s


### Downsampling обучение

<div class="alert alert-block alert-info">
<b>Комментарий:</b> Теперь попробуем методику downsampling
</div>

In [18]:
under = RandomUnderSampler(sampling_strategy = 0.125, random_state=12345)

In [19]:
f1_score_log_r_downsampling = []
m_log_r_downsampling = LogisticRegression(random_state=12345)
pipeline_log_r_downsampling = Pipeline([('tfid', count_tf_idf_train), ('downsampling', under), ('model', m_log_r_downsampling)])
for train, test in stratified_kfold.split(features_train,target_train):
    model = pipeline_log_r_downsampling.fit(features_train.iloc[train],target_train.iloc[train])
    prediction = model.predict(features_train.iloc[test])
    f1_score_log_r_downsampling.append(f1_score(target_train.iloc[test], prediction))
print("f1: ",np.mean(f1_score_log_r_downsampling))

f1:  0.7276344957705805


In [22]:
%%time
m_lgb_downsampling = lgb.LGBMClassifier(random_state=12345)
pipeline_lgb_downsampling = Pipeline([('tfid', count_tf_idf_train), ('downsampling', under), ('model', m_lgb_upsampling)])
f1_score_lgb_downsampling = []
for train, test in stratified_kfold.split(features_train,target_train):
    model = pipeline_lgb_downsampling.fit(features_train.iloc[train],target_train.iloc[train])
    prediction = model.predict(features_train.iloc[test])
    f1_score_lgb_downsampling.append(f1_score(target_train.iloc[test], prediction))
print("f1: ",np.mean(f1_score_lgb_downsampling))

f1:  0.7459693758423221
CPU times: user 12min 32s, sys: 255 ms, total: 12min 32s
Wall time: 12min 41s


## Выводы

<div class="alert alert-block alert-info">
<b>Комментарий:</b> Лучший результат показала LogisticRegression c upsampling, поэтому дальше протестируем ее на тестовой выборке.
</div>

In [23]:
pipeline_log_r_upsampling.fit(features_train, target_train)
print('Test f1 score:',f1_score(target_test,pipeline_log_r_upsampling.predict(features_test)))

Test f1 score: 0.7790790951387774


<div class="alert alert-block alert-info">
<b>Выводы:</b> Таким образом, мы получили необходимое значение метрики F1 на тестовой выборке, пусть и очень близкое с нижней границы допустимого. Все благодрая upsampling. Также следует понимать, что в нашем обучении мы использовали лишь TF-IDF, что учитывает частоту встречаемости слова в отдельном корпусе и во всем тексте, когда существуют Embeddings, которые также учитывают смысл, контекст и свойства слов. Пользуясь этой методикой, вероятнее всего, показатели f1 были бы выше. 
</div>